Intro: Trying to understand the structure of a 3x3 convolution kernel by looking at all of the possible 3x3 spatial slices is somewhat like trying to guess what an full image looks like from being shown all the 3x3 chunks of it in random order.

In [ ]:
#load the data
from keras.datasets import cifar10 as dset

(x_train, y_train), (x_test, y_test) = dset.load_data()

#normalize to lie in -1 to 1
x_train = (x_train-127.5)/128
x_test = (x_test-127.5)/128

y_train = y_train.squeeze()
y_test = y_test.squeeze()

n_categories = len(np.unique(y_train))

y_train_onehot = keras.utils.to_categorical(y_train, n_categories)
y_test_onehot = keras.utils.to_categorical(y_test, n_categories)

from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense

def make_net():
    input_img = keras.layers.Input(x_train.shape[1:])
    
    x = input_img
    x = keras.layers.GaussianNoise(0.5/128.0)(x)
    
    x = Conv2D(32, (3, 3), activation="relu", padding="same")(x)
    x = Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.3)(x)
    
    x = Conv2D(128, (3, 3), activation="relu", padding="same")(x)

    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.3)(x)
    
    x = Conv2D(256, (3, 3), activation="relu", padding="same")(x)
    
    x = keras.layers.GlobalMaxPooling2D()(x)
    
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.3)(x)
    probs = Dense(10, activation="softmax")(x)

    model = keras.models.Model(input_img, probs)
    
    return model

model = make_net()
model.compile(loss="categorical_crossentropy", optimizer="adadelta", metrics=["accuracy"])